In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import scipy 
from scipy import spatial

from itertools import combinations


In [4]:
np.random.seed(567)
times = np.sort(np.random.rand(10)*8)
times

array([2.41659224, 2.43825314, 2.75009709, 4.38612764, 5.44920096,
       5.64514133, 7.00984874, 7.62426376, 7.71766955, 7.95108611])

In [5]:
filtershere = np.array(['i','r','g']*5)
np.random.shuffle(filtershere)
filtershere

array(['i', 'i', 'i', 'r', 'g', 'g', 'r', 'i', 'g', 'i', 'g', 'r', 'r',
       'g', 'r'], dtype='<U1')

In [6]:
Detect = {"filters": filtershere}
Detect

{'filters': array(['i', 'i', 'i', 'r', 'g', 'g', 'r', 'i', 'g', 'i', 'g', 'r', 'r',
        'g', 'r'], dtype='<U1')}

In [7]:
dTs = abs(times.reshape(1, len(times)) - times.reshape(len(times), 1))

In [8]:
dTs.shape

(10, 10)

In [10]:
for i in range(dTs.shape[0]):
  #print(i, dTs[i])
  _tmp = np.where(dTs[i] < 2)[0]
  groupsOfThree = np.array(list(combinations(_tmp[i:],3)))
  for g3 in groupsOfThree:
    if g3[0] == i and len(np.unique(Detect["filters"][g3])) == 2:
        print(i, g3, Detect["filters"][g3])
      # calculate dM, color
      # get indices for filter 1 and filter 2
      #  get indices for dT1 and dT2
    

0 [0 1 3] ['i' 'i' 'r']
0 [0 2 3] ['i' 'i' 'r']
1 [1 2 3] ['i' 'i' 'r']
3 [3 4 5] ['r' 'g' 'g']


In [219]:
for i in range(dTs.shape[0]):
  #print(i, dTs[i])
  _tmp = np.where(dTs[i][i:] < 2)[0]
  groupsOfThree = np.array(list(combinations(_tmp+i,3)))
  for g3 in groupsOfThree:
    if g3[0] == i and len(np.unique(Detect["filters"][g3])) == 2:
      filters = Detect["filters"][g3]
      print(i, g3, filters)
      # calculate dM, color
      # get indices for filter 1 and filter 2
      #  get indices for dT1 and dT2
    

0 [0 1 3] ['i' 'i' 'r']
0 [0 2 3] ['i' 'i' 'r']
1 [1 2 3] ['i' 'i' 'r']
3 [3 4 5] ['r' 'g' 'g']
4 [4 5 6] ['g' 'g' 'r']
6 [6 7 9] ['r' 'i' 'i']
7 [7 8 9] ['i' 'g' 'i']


In [215]:
index0, index1 = np.where( ( abs(dTs)<2 ) ) #The time differences should be within 32 hours (2 nights)
print(index0)
print(index1)

[0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3 3 3 4 4 4 4 5 5 5 5 5 6 6 6 6 6 6 7 7 7 7
 7 8 8 8 8 9 9 9 9]
[0 1 2 3 0 1 2 3 0 1 2 3 0 1 2 3 4 5 3 4 5 6 3 4 5 6 7 4 5 6 7 8 9 5 6 7 8
 9 6 7 8 9 6 7 8 9]


In [11]:
%%timeit
index0, index1 = np.where( ( abs(dTs)<2 ) ) #The time differences should be within 32 hours (2 nights)
for ii in range(dTs.shape[0]):          
    groupsOfThrees = [ [ii] + list(jj) for jj in list(combinations( index1[ (index0==ii) * (index1>ii) ], 2)) ]    
    for g3 in groupsOfThrees:
        if len(np.unique(Detect["filters"][g3])) == 2:
            filters = Detect["filters"][g3]
            # print(ii, g3, filters)

729 µs ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%%timeit
for i in range(dTs.shape[0]):
    #print(i, dTs[i])
    _tmp = np.where(dTs[i][i:] < 2)[0]
    groupsOfThree = np.array(list(combinations(_tmp+i,3)))
    for g3 in groupsOfThree:
        if g3[0] == i and len(np.unique(Detect["filters"][g3])) == 2:
            filters = Detect["filters"][g3]
            # print(i, g3, filters)
            # calculate dM, color
            # get indices for filter 1 and filter 2
            #  get indices for dT1 and dT2
    

783 µs ± 8.17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
%%timeit
index0, index1 = np.where( ( abs(dTs)<2 ) ) 

for ii in range(dTs.shape[0]):        
    groupsOfThree = [ [ii] + list(jj) for jj in list(combinations( index1[ (index0==ii) * (index1>ii) ], 2)) ]    
    for g3 in groupsOfThree:
        if len(np.unique(Detect["filters"][g3])) == 2:
            filters = Detect["filters"][g3]
            # print(ii, g3, filters)

729 µs ± 8.57 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [221]:
dTindex0, dTindex1 = np.where( ( abs(dTs)<2 ) ) 

In [195]:
%%timeit

for ii in range(dTs.shape[0]):        
    groupsOfThree = np.array([ [ii] + list(jj) for jj in list(combinations( dTindex0[ (dTindex0==ii) * (dTindex1>ii) ], 2)) ])    
    for g3 in groupsOfThree:
        filters = Detect["filters"][g3]
        if len(np.unique(filters)) != 2:
            continue
        # print(ii, g3, filters)

10000 loops, best of 5: 174 µs per loop


In [204]:
%%timeit

for ii in range(dTs.shape[0]):        
    groupsOfThree = np.array(list(combinations( dTindex0[ (dTindex0==ii) * (dTindex1>=ii) ], 3)))
    for g3 in groupsOfThree:
        filters = Detect["filters"][g3]
        if g3[0]!=ii or len(np.unique(filters)) != 2:
            continue
        # print(ii, g3, filters)

10000 loops, best of 5: 193 µs per loop


In [226]:
for ii in range(dTs.shape[0]):        
    groupsOfThree = np.array([ [ii] + list(jj) for jj in list(combinations( dTindex0[ (dTindex0==ii) * (dTindex1>ii) ], 2)) ])    
    for g3 in groupsOfThree:
        filters = Detect["filters"][g3]
        if len(np.unique(filters)) != 2:
            continue
        print(ii, g3, filters)

In [225]:
print(ii, g3, filters)

9 [7 7 7] ['i' 'i' 'i']


In [223]:
for ii in range(dTs.shape[0]):        
    groupsOfThree = np.array(list(combinations( dTindex0[ (dTindex0==ii) * (dTindex1>=ii) ], 3)))
    for g3 in groupsOfThree:
        filters = Detect["filters"][g3]
        if g3[0]!=ii or len(np.unique(filters)) != 2:
            continue
        print(ii, g3, filters)

In [224]:
Detect

{'filters': array(['i', 'i', 'i', 'r', 'g', 'g', 'r', 'i', 'g', 'i', 'g', 'r', 'r',
        'g', 'r'], dtype='<U1')}

In [ ]:
%%timeit

for ii in range(dTs.shape[0]): 
    
    for jj in dTindex1[ (dTindex0==ii) * (dTindex1>ii) ]:
        
        for kk in dTindex1[ (dTindex0==ii) * (dTindex1>jj) ]:
            
            #subDTs = dTs[ [[ii],[jj],[kk]], [ii,jj,kk] ]
            
            #Determine the corresponding bands of the 3 observationss
            BandA = Detect['filters'][ii]
            BandB = Detect['filters'][jj]
            BandC = Detect['filters'][kk]
            
            Bands = np.array([BandA, BandB, BandC])
            BandsUnique = np.unique(Bands)
            
            if len(BandsUnique) != 2: #If the observations have 1 or 3 bands, continue
                continue
            # print(ii, [ii,jj,kk], Bands)

In [194]:
np.array(list(combinations([1,2,3,4,5],3)))

array([[1, 2, 3],
       [1, 2, 4],
       [1, 2, 5],
       [1, 3, 4],
       [1, 3, 5],
       [1, 4, 5],
       [2, 3, 4],
       [2, 3, 5],
       [2, 4, 5],
       [3, 4, 5]])